# Calculate post-TT dM/M0 and CBF time series for every ROI

Calculate the perfusion (dM/M0) time series  (i.e. vs PLD) for each ROI and filter only PLDs that are longer than the arterial transit time. Then calculate CBF using the white paper model, substituting assumed vs calculated T1 of arterial blood (aka T1a) parameter values. The best T1blood value should provide a flat response for PLD>ATT. That is because once the label reaches the voxel (PLD>TT) it just sits there and relaxes with T1b, so adjusting the CBF calculation for PLD with the correct T1a (arterial blood T1) should result in the same CBF regardless of PLD.


In [1]:
import os
import sys
import pandas as pd
import numpy as np
import subprocess
import glob
import flywheel

local_dir = '../data/subjects'

In [2]:
def refine_list(lst):
    '''
    creates a usable list of floats from the fsl output string

    :param lst: fsl output string
    :return: list of floats
    
    '''
    result_list = str(lst).split(' ')
    result_list[0] = result_list[0].replace("b'", "")
    result_list = result_list[: len(result_list) - 1] # remove extra text
    result_list = [float(e) for e in result_list]
    
    return(result_list)
    

In [21]:
cols = ['session_id', 'gray_matter','left_acc', 'right_acc', 'left_insula', 'right_insula', 'left_caudate', 'right_caudate', 'left_putamen', 'right_putamen']
df = pd.DataFrame(columns=cols)
for root, dirs, files in os.walk(local_dir, topdown=False):
    pw_maps = [f for f in files if 'dMM0' in f]
    if len(pw_maps) > 0:
        for pw_map in pw_maps:
            ses_name = pw_map.split('dMM0')[0][:-1] # drop the last underscore
            pw_map_path = os.path.join(root, pw_map)
            roi_dir = os.path.join(root, ses_name + "rois")
            
            for mask in glob.glob(roi_dir+"/*roimask.nii.gz"):
                cmd1="fslmeants --transpose -i '{}' -m '{}'".format(pw_map_path, mask)
                result = refine_list(subprocess.check_output(cmd1, shell=True))
                if 'left_acc' in mask:
                    left_acc = result
                elif 'right_acc' in mask:
                    right_acc = result
                elif 'left_insula' in mask:
                    left_insula = result
                elif 'right_insula' in mask:
                    right_insula = result
                elif 'left_caudate' in mask:
                    left_caudate = result
                elif 'right_caudate' in mask:
                    right_caudate = result
                elif 'left_putamen' in mask:
                    left_putamen = result
                elif 'right_putamen' in mask:
                    right_putamen = result
                elif 'GM' in mask:
                    gray_matter = result

            
            
            df.loc[len(df.index)] = [ses_name, gray_matter, left_acc, right_acc, left_insula, right_insula, left_caudate, right_caudate, left_putamen, right_putamen]

df.head()

,session_id,gray_matter,left_acc,right_acc,left_insula,right_insula,left_caudate,right_caudate,left_putamen,right_putamen
0,sub-HC015_ses-MR_FOLLOWUP,"[0.027081, 0.026786, 0.023863, 0.014177, 0.015...","[0.014385, 0.015414, 0.01481, 0.010419, 0.0092...","[0.041746, 0.025004, 0.023513, 0.014687, 0.016...","[0.037308, 0.024532, 0.01983, 0.016345, 0.0148...","[0.011922, 0.012569, 0.0109, 0.008592, 0.00718...","[0.00644, 0.007332, 0.007101, 0.006104, 0.0049...","[0.033342, 0.015873, 0.015562, 0.011294, 0.010...","[0.017936, 0.015135, 0.012558, 0.009225, 0.008...","[0.011533, 0.010596, 0.00914, 0.005589, 0.0052..."
1,sub-HC025_ses-MR_BASELINE,"[0.027081, 0.026786, 0.023863, 0.014177, 0.015...","[0.014385, 0.015414, 0.01481, 0.010419, 0.0092...","[0.041746, 0.025004, 0.023513, 0.014687, 0.016...","[0.037308, 0.024532, 0.01983, 0.016345, 0.0148...","[0.011922, 0.012569, 0.0109, 0.008592, 0.00718...","[0.00644, 0.007332, 0.007101, 0.006104, 0.0049...","[0.033342, 0.015873, 0.015562, 0.011294, 0.010...","[0.017936, 0.015135, 0.012558, 0.009225, 0.008...","[0.011533, 0.010596, 0.00914, 0.005589, 0.0052..."
2,sub-HC025_ses-MR_FOLLOWUP,"[0.027081, 0.026786, 0.023863, 0.014177, 0.015...","[0.014385, 0.015414, 0.01481, 0.010419, 0.0092...","[0.041746, 0.025004, 0.023513, 0.014687, 0.016...","[0.037308, 0.024532, 0.01983, 0.016345, 0.0148...","[0.011922, 0.012569, 0.0109, 0.008592, 0.00718...","[0.00644, 0.007332, 0.007101, 0.006104, 0.0049...","[0.033342, 0.015873, 0.015562, 0.011294, 0.010...","[0.017936, 0.015135, 0.012558, 0.009225, 0.008...","[0.011533, 0.010596, 0.00914, 0.005589, 0.0052..."
3,sub-HC005_ses-MR_BASELINE,"[0.027081, 0.026786, 0.023863, 0.014177, 0.015...","[0.014385, 0.015414, 0.01481, 0.010419, 0.0092...","[0.041746, 0.025004, 0.023513, 0.014687, 0.016...","[0.037308, 0.024532, 0.01983, 0.016345, 0.0148...","[0.011922, 0.012569, 0.0109, 0.008592, 0.00718...","[0.00644, 0.007332, 0.007101, 0.006104, 0.0049...","[0.033342, 0.015873, 0.015562, 0.011294, 0.010...","[0.017936, 0.015135, 0.012558, 0.009225, 0.008...","[0.011533, 0.010596, 0.00914, 0.005589, 0.0052..."
4,sub-HC005_ses-MR_FOLLOWUP,"[0.027081, 0.026786, 0.023863, 0.014177, 0.015...","[0.014385, 0.015414, 0.01481, 0.010419, 0.0092...","[0.041746, 0.025004, 0.023513, 0.014687, 0.016...","[0.037308, 0.024532, 0.01983, 0.016345, 0.0148...","[0.011922, 0.012569, 0.0109, 0.008592, 0.00718...","[0.00644, 0.007332, 0.007101, 0.006104, 0.0049...","[0.033342, 0.015873, 0.015562, 0.011294, 0.010...","[0.017936, 0.015135, 0.012558, 0.009225, 0.008...","[0.011533, 0.010596, 0.00914, 0.005589, 0.0052..."


In [23]:
# example
list(df.loc[df['session_id']=='sub-HC001_ses-MR_BASELINE', 'left_acc'])[0]

[0.014385, 0.015414, 0.01481, 0.010419, 0.009204, 0.008159, 0.00433]

## Select time points based on transit time for that region
Take the time series list from the above calculation and exclude time points (i.e. PLDs) that happen before ATT (i.e. when all the blood has arrived). The PLDs in this experiment are [1, 1.22, 1.48, 1.78, 2.15,x, y]. We can determine which to keep based on how long the transit-time was using a decision tree. E.g. if the transit time was between the first (1s) and second (1.22s) delays, then we eliminate the first from the data. We do this by adding a NaN at the end, so that the resulting time series is PLDs after ATT (i.e. first PLD after, second PLD after) rather than the absolute (first (1s), second (1.22s), etc.)

In [24]:
# Check the maximum transit-time
tt_df = pd.read_csv("regionals_tts2.csv")
np.max(tt_df)

session_id       sub-HC028_ses-MR_FOLLOWUP
gray_matter                    1793.280349
left_acc                       1885.564396
right_acc                      1752.977169
left_insula                    1533.890533
right_insula                   1404.397273
left_caudate                   1524.748148
right_caudate                  1959.006284
left_putamen                   1624.468193
right_putamen                   1951.27004
dtype: object

It looks like the maximum transit time in the dataset is not more than the fifth PLD (2.15s). So there should be at least 3 points in every time series.

In [25]:
refined_df = df.copy()

for row, row in df.iterrows(): # iterate over rows
    row_list = [row.session_id] 
    ses_id = row.session_id
    for c, value in row.items():
        if type(value) == list:
            tt = float(tt_df.loc[tt_df['session_id']==row.session_id, c])/1000 # originally in ms, but we want s
            
            # extract post-TT values
            if tt < 1:
                dms = value[0:7] # all 7
            elif 1 < tt < 1.22:
                dms = value[1:7] + [np.nan]
            elif 1.22 < tt < 1.48:
                dms = value[2:7] + [np.nan]*2
            elif 1.48 < tt < 1.78:
                dms = value[3:7] + [np.nan]*3
            elif 1.78 < tt < 2.15:
                dms = value[4:7] + [np.nan]*4 # 5,6,7 see above comment
            else:
                print("??")

            row_list.append(dms)

    refined_df.loc[len(refined_df.index)] = row_list
    
refined_df = refined_df.tail(50) # only get the newly created rows
refined_df = refined_df.reset_index(drop=True)
refined_df.head()

,session_id,gray_matter,left_acc,right_acc,left_insula,right_insula,left_caudate,right_caudate,left_putamen,right_putamen
0,sub-HC015_ses-MR_FOLLOWUP,"[0.026786, 0.023863, 0.014177, 0.015461, 0.012...","[0.015414, 0.01481, 0.010419, 0.009204, 0.0081...","[0.025004, 0.023513, 0.014687, 0.016256, 0.012...","[0.024532, 0.01983, 0.016345, 0.014821, 0.0140...","[0.012569, 0.0109, 0.008592, 0.007185, 0.00729...","[0.00644, 0.007332, 0.007101, 0.006104, 0.0049...","[0.015562, 0.011294, 0.010482, 0.009723, 0.004...","[0.015135, 0.012558, 0.009225, 0.00866, 0.0076...","[0.00914, 0.005589, 0.005263, 0.005307, 0.0039..."
1,sub-HC025_ses-MR_BASELINE,"[0.026786, 0.023863, 0.014177, 0.015461, 0.012...","[0.015414, 0.01481, 0.010419, 0.009204, 0.0081...","[0.025004, 0.023513, 0.014687, 0.016256, 0.012...","[0.024532, 0.01983, 0.016345, 0.014821, 0.0140...","[0.012569, 0.0109, 0.008592, 0.007185, 0.00729...","[0.007332, 0.007101, 0.006104, 0.004953, 0.005...","[0.015873, 0.015562, 0.011294, 0.010482, 0.009...","[0.015135, 0.012558, 0.009225, 0.00866, 0.0076...","[0.00914, 0.005589, 0.005263, 0.005307, 0.0039..."
2,sub-HC025_ses-MR_FOLLOWUP,"[0.023863, 0.014177, 0.015461, 0.01241, 0.0085...","[0.015414, 0.01481, 0.010419, 0.009204, 0.0081...","[0.025004, 0.023513, 0.014687, 0.016256, 0.012...","[0.024532, 0.01983, 0.016345, 0.014821, 0.0140...","[0.012569, 0.0109, 0.008592, 0.007185, 0.00729...","[0.007332, 0.007101, 0.006104, 0.004953, 0.005...","[0.015562, 0.011294, 0.010482, 0.009723, 0.004...","[0.015135, 0.012558, 0.009225, 0.00866, 0.0076...","[0.00914, 0.005589, 0.005263, 0.005307, 0.0039..."
3,sub-HC005_ses-MR_BASELINE,"[0.023863, 0.014177, 0.015461, 0.01241, 0.0085...","[0.015414, 0.01481, 0.010419, 0.009204, 0.0081...","[0.023513, 0.014687, 0.016256, 0.012321, 0.008...","[0.024532, 0.01983, 0.016345, 0.014821, 0.0140...","[0.012569, 0.0109, 0.008592, 0.007185, 0.00729...","[0.007332, 0.007101, 0.006104, 0.004953, 0.005...","[0.015562, 0.011294, 0.010482, 0.009723, 0.004...","[0.015135, 0.012558, 0.009225, 0.00866, 0.0076...","[0.005589, 0.005263, 0.005307, 0.003921, nan, ..."
4,sub-HC005_ses-MR_FOLLOWUP,"[0.023863, 0.014177, 0.015461, 0.01241, 0.0085...","[0.015414, 0.01481, 0.010419, 0.009204, 0.0081...","[0.023513, 0.014687, 0.016256, 0.012321, 0.008...","[0.024532, 0.01983, 0.016345, 0.014821, 0.0140...","[0.012569, 0.0109, 0.008592, 0.007185, 0.00729...","[0.007332, 0.007101, 0.006104, 0.004953, 0.005...","[0.011294, 0.010482, 0.009723, 0.004524, nan, ...","[0.012558, 0.009225, 0.00866, 0.007613, 0.0037...","[0.005589, 0.005263, 0.005307, 0.003921, nan, ..."


## Pull Hematocrit data from Flywheel
The hematocrit data is essential for calculating T1a. It lives on Flywheel in each subject's custom info. 

In [28]:
fw = flywheel.Client()
project_id = '5e50277c6dea314fa72a7440'
project = fw.get(project_id)

# initialize target columns with NaNs
refined_df['Hb'] = np.nan
refined_df['T1a'] = np.nan

sessions = [fw.get_session(x.id) for x in fw.get_project_sessions(project_id)]

for ses in sessions:
    subject = fw.get(ses.parents['subject'])
    session_id = "sub-{}_ses-{}".format(subject.label, ses.label) # NOTE this is not the Flywheel definition of 'id'

    if subject.info:
        hb = subject.info['Hb']
        t1a = hb / 3 #??  conversion equation
        refined_df.loc[refined_df['session_id']==session_id, 'Hb'] = hb
        refined_df.loc[refined_df['session_id']==session_id, 'T1a'] = t1a 

refined_df.to_csv('refined_dM2.csv', index=False)
refined_df.head()

,session_id,gray_matter,left_acc,right_acc,left_insula,right_insula,left_caudate,right_caudate,left_putamen,right_putamen,Hb,T1a
0,sub-HC015_ses-MR_FOLLOWUP,"[0.026786, 0.023863, 0.014177, 0.015461, 0.012...","[0.015414, 0.01481, 0.010419, 0.009204, 0.0081...","[0.025004, 0.023513, 0.014687, 0.016256, 0.012...","[0.024532, 0.01983, 0.016345, 0.014821, 0.0140...","[0.012569, 0.0109, 0.008592, 0.007185, 0.00729...","[0.00644, 0.007332, 0.007101, 0.006104, 0.0049...","[0.015562, 0.011294, 0.010482, 0.009723, 0.004...","[0.015135, 0.012558, 0.009225, 0.00866, 0.0076...","[0.00914, 0.005589, 0.005263, 0.005307, 0.0039...",14.8,4.933333
1,sub-HC025_ses-MR_BASELINE,"[0.026786, 0.023863, 0.014177, 0.015461, 0.012...","[0.015414, 0.01481, 0.010419, 0.009204, 0.0081...","[0.025004, 0.023513, 0.014687, 0.016256, 0.012...","[0.024532, 0.01983, 0.016345, 0.014821, 0.0140...","[0.012569, 0.0109, 0.008592, 0.007185, 0.00729...","[0.007332, 0.007101, 0.006104, 0.004953, 0.005...","[0.015873, 0.015562, 0.011294, 0.010482, 0.009...","[0.015135, 0.012558, 0.009225, 0.00866, 0.0076...","[0.00914, 0.005589, 0.005263, 0.005307, 0.0039...",15.9,5.300000
2,sub-HC025_ses-MR_FOLLOWUP,"[0.023863, 0.014177, 0.015461, 0.01241, 0.0085...","[0.015414, 0.01481, 0.010419, 0.009204, 0.0081...","[0.025004, 0.023513, 0.014687, 0.016256, 0.012...","[0.024532, 0.01983, 0.016345, 0.014821, 0.0140...","[0.012569, 0.0109, 0.008592, 0.007185, 0.00729...","[0.007332, 0.007101, 0.006104, 0.004953, 0.005...","[0.015562, 0.011294, 0.010482, 0.009723, 0.004...","[0.015135, 0.012558, 0.009225, 0.00866, 0.0076...","[0.00914, 0.005589, 0.005263, 0.005307, 0.0039...",15.9,5.300000
3,sub-HC005_ses-MR_BASELINE,"[0.023863, 0.014177, 0.015461, 0.01241, 0.0085...","[0.015414, 0.01481, 0.010419, 0.009204, 0.0081...","[0.023513, 0.014687, 0.016256, 0.012321, 0.008...","[0.024532, 0.01983, 0.016345, 0.014821, 0.0140...","[0.012569, 0.0109, 0.008592, 0.007185, 0.00729...","[0.007332, 0.007101, 0.006104, 0.004953, 0.005...","[0.015562, 0.011294, 0.010482, 0.009723, 0.004...","[0.015135, 0.012558, 0.009225, 0.00866, 0.0076...","[0.005589, 0.005263, 0.005307, 0.003921, nan, ...",12.2,4.066667
4,sub-HC005_ses-MR_FOLLOWUP,"[0.023863, 0.014177, 0.015461, 0.01241, 0.0085...","[0.015414, 0.01481, 0.010419, 0.009204, 0.0081...","[0.023513, 0.014687, 0.016256, 0.012321, 0.008...","[0.024532, 0.01983, 0.016345, 0.014821, 0.0140...","[0.012569, 0.0109, 0.008592, 0.007185, 0.00729...","[0.007332, 0.007101, 0.006104, 0.004953, 0.005...","[0.011294, 0.010482, 0.009723, 0.004524, nan, ...","[0.012558, 0.009225, 0.00866, 0.007613, 0.0037...","[0.005589, 0.005263, 0.005307, 0.003921, nan, ...",12.2,4.066667


## Calculate CBF


In [8]:
def calc_cbf_ts(dMM0_list, T1blood):
    '''
    calculates CBF values from list of DM/M0 values

    :param dMM0_list:  the list of dM/M0 values
    :param T1blood: the assumed (1.65) or calculated T1 relaxation time of blood
    :return: list of CBF values
    
    '''
    
    plds = [1, 1.22, 1.48, 1.78, 2.15, 2.63, 3.32] # post-labeling durations
    cbf_ts = []
    
    # set the parameters
    lmbda = 0.9
    alpha = 0.85
    tau = 3.5 # labeling duration
    
    denominator = 2*alpha*T1blood*(1-np.exp(-tau/T1blood)) # only need to calculate once                         
                                   
    for i in range(len(dMM0_list)-1):
        # calculate CBF factor based on corresponding PLD value
        pld = plds[i]                                   
        numerator = 6000 * lmbda * np.exp(pld/T1blood)
        factor = numerator/denominator                           
        # multiply by deltaM over M0 to get the CBF value for this PLD                          
        cbf = dMM0_list[i] * factor
        cbf_ts.append(cbf) 
                                
    return cbf_ts
    

In [9]:
cbf_df = refined_df.copy()
for row, row in refined_df.iterrows(): #iterate over rows
    row_list = [row.session_id]
    for c, value in row.items():
        if type(value) == list:
            cbf_ts = calc_cbf_ts(value, 1.65) 
            row_list.append(cbf_ts)
    
    
    row_list.append(row.Hb)
    row_list.append(row.T1a)
#     print(row_list)
    cbf_df.loc[len(cbf_df.index)] = row_list

cbf_df = cbf_df.tail(50) # only get the newly created rows
cbf_df = cbf_df.reset_index(drop=True)
# cbf_df.to_csv('cbf_ts.csv', index=False)
cbf_df.head()

,session_id,left_acc,right_acc,left_insula,right_insula,left_caudate,right_caudate,left_putamen,right_putamen,Hb,T1a
0,sub-HC015_ses-MR_FOLLOWUP_,"[89.49640375770703, 77.93653558038963, 98.1893...","[163.29013637803425, 146.92022045389615, 118.0...","[138.88603613745872, 130.55400616270558, 76.23...","[94.49270821050972, 78.56424312739216, 64.4619...","[57.44547158174251, 43.5363292818849, 57.06526...","[65.44517253145466, 40.29240998788632, 40.8988...","[95.68364113385033, 81.7485989315292, 71.19883...","[57.910617100623014, 70.82557125230234, 45.243...",14.8,4.933333
1,sub-HC025_ses-MR_BASELINE_,"[86.48498751047204, 77.79449956610438, 82.4842...","[142.28641027548173, 118.8383258876288, 125.93...","[116.12197483845326, 94.8067486319472, 120.561...","[77.91588428764754, 64.49809590624014, 76.3856...","[35.92447158319357, 26.313317098070403, 31.742...","[45.026888202665546, 49.88213185559682, 61.726...","[90.09788503212148, 73.30432879192554, 89.5858...","[41.947304077663574, 51.636963903056504, 47.49...",15.9,5.300000
2,sub-HC025_ses-MR_FOLLOWUP_,"[70.4895954862105, 80.09456663614291, 48.98741...","[121.81198769441393, 123.87373168438647, 82.60...","[110.6124063648169, 123.49802351756742, 85.981...","[72.82735270610132, 77.31799164721193, 61.0773...","[34.64933128143494, 40.636045506318375, 19.025...","[48.73602272822131, 36.37038327084849, 22.0505...","[78.89028395219896, 85.6843710693043, 61.84439...","[58.30759474173655, 44.081564304463754, 32.579...",15.9,5.300000
3,sub-HC005_ses-MR_BASELINE_,"[75.11699142403897, 72.80491183847097, 86.3836...","[126.96467727856435, 130.90222348805005, 85.46...","[144.72040449927886, 118.63214457656954, 138.0...","[77.71539052950939, 70.88055293525146, 74.7657...","[46.73509502200258, 35.09664094919368, 45.3293...","[60.94208272367183, 77.99151726333875, 32.7619...","[94.30424407785986, 74.5185076237188, 86.51242...","[58.64041438024588, 59.435199268007885, 29.785...",12.2,4.066667
4,sub-HC005_ses-MR_FOLLOWUP_,"[56.28661765970401, 62.381301112698694, 57.462...","[107.05163722028338, 75.45319623385399, 118.59...","[92.4476718775006, 80.26409349190273, 120.5992...","[68.80544791785005, 55.2199369085747, 49.34142...","[36.0728369642158, 32.0359939316922, 23.906373...","[38.58301881610544, 66.7844175555414, 70.88773...","[48.83225973212762, 46.71152147219707, 58.8084...","[33.52656623586131, 43.101057625204305, 30.305...",12.2,4.066667


### Refine CBF ts data based on transit time, like for dM/M0 data
Again, take the time series list from the above calculation and exclude time points (i.e. PLDs) that happen before ATT (i.e. when all the blood has arrived).

In [10]:
tt_df = pd.read_csv("regionals_tts2.csv")
refined_cbf_df = cbf_df.copy()

for row, row in cbf_df.iterrows(): #iterate over rows
    row_list = [row.session_id]
    for c, value in row.items():
        if type(value) == list:
            
            tt = float(tt_df.loc[tt_df['session_id']==row.session_id, c])/1000
 
            # extract post-TT values
            if tt < 1:
                dms = value[0:7]
            elif 1 < tt < 1.22:
                dms = value[1:7] + [0]
            elif 1.22 < tt < 1.48:
                dms = value[2:7] + [0]*2
            elif 1.48 < tt < 1.78:
                dms = value[3:7] + [0]*3
            elif 1.78 < tt < 2.15:
                dms = value[4:7] + [0]*4
            else:
                print("??")

            row_list.append(dms)
            
    row_list.append(row.Hb)
    row_list.append(row.T1a)
    refined_cbf_df.loc[len(refined_cbf_df.index)] = row_list
    
refined_cbf_df = refined_cbf_df.tail(50) # only get the newly created rows
refined_cbf_df.reset_index(drop=True)
refined_cbf_df.to_csv('refined_cbf.csv', index=False)